# Notebook for implementing ISCE2 interferometry to form Interferograms, and MintPy SBAS Time-Series analysis
This notebook will:
1. Find scenes from ASF Vertex using the asf_search library
2. Download said scenes
3. Use scenes to form interferograms using ISCE2
4. Utilize the closure phase to improve phase unwrapping
5. Create SBAS Time-Series of deformation using the closure phase corrected interferograms
6. Relate non-zero closure phase to surface vegatation and mositure changes using NDVI and EO moisture products
7. Report on erosion driver events and show relation to deformation and insar velocity from corrected SBAS time-series 

# To-DO
1. Get notebook working first with manually installed SLC images
2. Build off 'mintpy_get_isceburst.ipynb' to build interactive for retrieving SLCs

https://nbviewer.org/github/isce-framework/isce2-docs/blob/master/Notebooks/UNAVCO_2020/TOPS/topsApp.ipynb

# First, some helpful functions

In [1]:
# importing needed libraries for the entire notebook in one go
# %matplotlib inline
# %matplotlib widget
import os
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from typing import List, Union
from itertools import combinations
import eof
import geemap
import csv
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, box


# only need these two if you plan on searching andrequesting for data from within this notebook
# in my opinion, while it may take longer, it is much easier to use the ASF Vertex Website
# that said, if you are monitoring a reoccuring site and just need more imagery, searching using the notebook would be best
import asf_search as asf
import hyp3_sdk as sdk

In [2]:
def project_dir(proj_name, operatingsys):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    proj_name = str
        string that contains the name of your project

    operatingsys = str
        'linux' or 'wsl', the linux operating system you are using will impact where data will be save
    """

    if operatingsys == 'linux':
        print('Creating Project Working Directory in your "~" directory')
        work_dir = os.path.join(os.path.expanduser('~'),f'{proj_name}_InSAR')
    elif operatingsys == 'wsl':
        print('Creating Project Working Directory on your Windows Desktop')
        work_dir = os.path.join('/mnt/c/Users/clayc/Desktop',f'{proj_name}_InSAR')

    #creates file on your desktop containing the work of this notebook
    os.makedirs(work_dir, exist_ok=True)
    
    # file inside work_dir for isce2 interferometry
    if_dir = os.path.join(work_dir,'interferometry')
    os.makedirs(if_dir, exist_ok=True)
    
    # file inside work_dir for mintpy time-series
    ts_dir = os.path.join(work_dir,'time_series')
    os.makedirs(ts_dir, exist_ok=True)
    

    xmls_dir = os.path.join(if_dir,'xmls')
    os.makedirs(xmls_dir, exist_ok=True)
    
    xmldirectories=[]
    for dir in ['topsApp', 'reference', 'secondary']:
        dirpath = os.path.join(xmls_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        xmldirectories.append(dirpath)

    ifdirectories=[]
    for dir in ['reference', 'secondary', 'orbits', 'work']:
        dirpath = os.path.join(if_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        ifdirectories.append(dirpath)

    tsdirectories=[]
    for dir in ['baseline', 'reference', 'merged', 'secondaries', 'mintpy']:
        dirpath = os.path.join(ts_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        if dir == 'merged':
                geomref_dir = os.path.join(dirpath,'geom_reference')
                os.makedirs(geomref_dir, exist_ok=True)

                interfer_dir = os.path.join(dirpath,'interferograms')
                os.makedirs(interfer_dir, exist_ok=True)

                tsdirectories.append(geomref_dir)
                tsdirectories.append(interfer_dir)

        tsdirectories.append(dirpath)

    return work_dir, ifdirectories, tsdirectories, xmldirectories

In [3]:
def get_triplets(slc_list):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    slc_zips = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """
    
    triplet_dict={}
    
    for i in range(len(slc_list) - 2):
        triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]))
        if i == 0:
            test_triplet = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]), (slc_list[i], slc_list[i+2]))
        else:
            continue
    return triplet_dict, test_triplet

In [4]:
# function to create the reference and secondary .xml files needed for ISCE2 interferometry
# function to write the 
def ref_sec_xml(slc_zips_list, slc_zips_dirs):
    for i in range(len(slc_zips_dirs)):
        for j, type in enumerate(['reference', 'secondary']):
            imset = ET.Element('component', name=type)
            safe = ET.SubElement(imset, 'property', name='safe').text = slc_zips_dirs[i]
            out_dir =ET.SubElement(imset, 'property', name='output directory').text = ifdirectories[j]
            orbit_dir =ET.SubElement(imset, 'property', name='orbit directory').text = ifdirectories[2]
            roi = ET.SubElement(imset, 'property', name='region of interest').text = str(isce_aoi)
            tree = ET.ElementTree(imset)
            tree.write(os.path.join(xmldirectories[j+1], f'{slc_zips_list[i][17:25]}{type[:3]}.xml'))

In [5]:
# function to create the topsApp.xml files for topsInSAR
def topsApp_xml(triplet_dict, do_iono):
    """
    do_iono = str
        'True' or 'False', does ionospheric correction of the interfoerograms
    """

    for key in triplet_dict:
        for pair in triplet_dict[key]:
            data = ET.Element('topsApp')
            tinsar = ET.SubElement(data, 'component', name='topsinsar')
            snsr_name = ET.SubElement(tinsar, 'property', name='Sensor name').text = 'SENTINEL1'
            ref_xmlcomp = ET.SubElement(tinsar, 'component', name='reference')
            xml_file = ET.SubElement(ref_xmlcomp, 'catalog').text = os.path.join(xmldirectories[1], f"{pair[0][17:25]}ref.xml")
            sec_xmlcomp = ET.SubElement(tinsar, 'component', name='secondary')
            xml_file = ET.SubElement(sec_xmlcomp, 'catalog').text = os.path.join(xmldirectories[2], f"{pair[1][17:25]}sec.xml")
            
            swathnums = ET.SubElement(tinsar, 'property', name='swaths').text=str([1,2])

            azi_looks = ET.SubElement(tinsar, 'property', name='azimuth looks').text = str(6)
            range_looks = ET.SubElement(tinsar, 'property', name='range looks').text = str(2)
            
            phasefilt = ET.SubElement(tinsar, 'property', name='filter strength').text = str(0.2)
            
            
            unwrap_yn = ET.SubElement(tinsar, 'property', name='do unwrap').text = str(True) # or False
            unwrap_name = ET.SubElement(tinsar, 'property', name='unwrapper name').text = 'snaphu_mcf' # grass, icu, snaphu, snaphu_mcf, downsample_snap
            
            do_iono_corr = ET.SubElement(tinsar, 'property', name='do ionosphere correction').text = do_iono
            apply_iono_corr = ET.SubElement(tinsar, 'property', name='apply ionosphere correction').text = do_iono

            if do_iono == 'True':
                #choose from: ['subband', 'rawion', 'grd2ion', 'filt_gaussian', 'ionosphere_shift', 'ion2grd', 'esd']
                start_iono_corr = ET.SubElement(tinsar, 'property', name='start ionosphere step').text = 'filt_gaussian'
                end_iono_corr = ET.SubElement(tinsar, 'property', name='end ionosphere step').text = 'esd'

                iono_height = ET.SubElement(tinsar, 'property', name='height of ionosphere layer in km').text = str(200.0)
                iono_filt = ET.SubElement(tinsar, 'property', name="apply polynomial fit before filtering ionosphere phase").text = str(True)
                max_iono_filt_wind_phase = ET.SubElement(tinsar, 'property', name="maximum window size for filtering ionosphere phase").text = str(200)
                min_iono_filt_wind_phase = ET.SubElement(tinsar, 'property', name="minimum window size for filtering ionosphere phase").text = str(100)
                max_iono_filt_wind_azishift = ET.SubElement(tinsar, 'property', name="maximum window size for filtering ionosphere azimuth shift").text = str(150)
                min_iono_filt_wind_azishift = ET.SubElement(tinsar, 'property', name="minimum window size for filtering ionosphere azimuth shift").text = str(75)

                #0: no correction
                #1: use mean value of a burst
                #2: use full burst
                correct_iono_azishift_err = ET.SubElement(tinsar, 'property', name='correct phase error caused by ionosphere azimuth shift').text = str(2)

            geocode = ET.SubElement(tinsar, 'property', name='geocode bounding box').text = str(isce_aoi)
            
            tree = ET.ElementTree(data)
            tree.write(os.path.join(xmldirectories[0], f"{pair[0][17:25]}_{pair[1][17:25]}topsApp.xml"))

# Establish working directroy and data paths

In [6]:
# Establish working directories, and locate Sentinel-1 SLC .zip files
proj_name = 'Harris_etal_2023'
work_dir, ifdirectories, tsdirectories, xmldirectories = project_dir(proj_name, 'linux')

Creating Project Working Directory in your "~" directory


In [7]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

triplet_dict, test_triplet = get_triplets(slc_zips_list)

In [8]:
# examine each triplet if you would like
# this shows the tuple that stores the string filenmem of the SLC .zip files
# in each triplet there will be 4 SLC pairs = 4 interferograms
# the first makes if_ij = the multilooked wrapped interferometric phase between SLC i and j
# the second makes if_jk = the multilooked wrapped interferometric phase between SLC j and k
# the third makes if_ki = the multilooked wrapped interferometric phase between SLC k and i
# where i, j, k are a time-series of SLC images
# choose a triplet to examine its pairs
# closure phase = if_ij+if_jk+if_ki

# triplet_choice = 'triple_1'
triplet_choice = f'triplet_{len(triplet_dict)}'  # uncomment to just check the last triplet in the dict

if_ij = triplet_dict[triplet_choice][0]
if_jk = triplet_dict[triplet_choice][1]
if_ki = triplet_dict[triplet_choice][2]

print()
print(f'Pair 1:{if_ij}')
print(f'Pair 2:{if_jk}')
print(f'Pair 3:{if_ki}')


Pair 1:('S1A_IW_SLC__1SDV_20180208T001821_20180208T001848_020508_023144_A39F.zip', 'S1A_IW_SLC__1SDV_20180220T001821_20180220T001848_020683_0236D8_9211.zip')
Pair 2:('S1A_IW_SLC__1SDV_20180220T001821_20180220T001848_020683_0236D8_9211.zip', 'S1A_IW_SLC__1SDV_20180328T001822_20180328T001849_021208_024782_46FD.zip')
Pair 3:('S1A_IW_SLC__1SDV_20180328T001822_20180328T001849_021208_024782_46FD.zip', 'S1A_IW_SLC__1SDV_20180208T001821_20180208T001848_020508_023144_A39F.zip')


# Get bbox for your area of interest in the SLC images, faster processing with less data

In [48]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')

## Add drawing tools
m.add_draw_control()

## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [ ]:
# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(m.draw_features[0].getInfo()['geometry']['coordinates'][0])]

# create POLYGON string to use when searching asf for imagery
isce_aoi_polygon = Polygon(vertices)

#create isce compatibile bbox
isce_aoi = [min([easting for northing, easting in vertices[:]]), max([easting for northing, easting in vertices[:]]), min([northing for northing, easting in vertices[:]]), max([northing for northing, easting in vertices[:]])]
print(isce_aoi)

In [9]:
isce_aoi = [29.813242, 30.089296, -93.544464, -93.309631]

# Get s-1 orbit files (.EOF) using sentineleof library

In [10]:
#below downloads orbit files for your S1 SLC imagery to orbits_dir created with project_dir
for i,slc in enumerate(slc_zips_list):
    eof.download.download_eofs(
        orbit_dts=slc_zips_dates[i],  # slc date in str YYYYMMDD format
        missions=['S1A', 'S1B'],        # gets both S1 missions, third was just launched (2024) so may need updating
        sentinel_file=slc,              # image name
        save_dir=ifdirectories[2],      # orbits_dir
        orbit_type='precise'            # can be 'precise' or 'restituted'
    )

# Create Sentinel-1 SLC .xml files
- One reference and one secondary for each image

In [11]:
ref_sec_xml(slc_zips_list, slc_zips_dirs)

# Create topsApp.xml file
- one for each interferogram in each triplet
- (N_triplets * 4) - (N_triplets - 1)   ??

In [12]:
topsApp_xml(triplet_dict, 'False') # stores these in xmldirectories[0]

In [13]:
# check to make sure all topsApp.xml files were created
# if the cell prints somehting, not all topsApp.xmls were created

topsAppxml_triplet_dict = {}
for key, triplets in triplet_dict.items():
    topsapppairs = (f'{triplets[0][0][17:25]}_{triplets[0][1][17:25]}topsApp.xml', 
             f'{triplets[1][0][17:25]}_{triplets[1][1][17:25]}topsApp.xml', 
             f'{triplets[2][0][17:25]}_{triplets[2][1][17:25]}topsApp.xml'
             )
    topsAppxml_triplet_dict[key] = topsapppairs


for i in range(1, len(triplet_dict)+1):
    topsapptrip = topsAppxml_triplet_dict[f'triplet_{i}']
    slctrip = triplet_dict[f'triplet_{i}']
    for j, slcpair in enumerate(slctrip):
        slcdate = f'{slcpair[0][17:25]}_{slcpair[1][17:25]}'
        topsappdate = topsapptrip[j][:-11]
        if slcdate != topsappdate:
            print(slcdate, topsappdate)

# topsApp interferometry (finally!)

In [14]:
# should import isce toolbox and print the version of it here
# if nothing prints or nothing imports, you need to reinstall isce2

import isce
isce.version.release_version

'2.6.3'

Below is the cell that performs the InSAR processing. Still need to work on a couple things:

1. ISCE2 CLI allows for running separate steps and saving pickles between steps, want to figure out how to do that in the IDE
2. Iterating through each .xml to create the interferograms
3. Saving XMLs to appropriate storage locations for MintPy time-series
4. runnings prep_isce.py from mintPy to prep the topsApp results

Need to look into the difference between topsStack and topsApp results, as Mintpy docs only specified topsStack. Presumably, topsStack stacks the interferograms in an easier to use format for mintpy, compared to the individual interferograms generated by topsApp

In [15]:
# this is used to change directory into the storage directory for all interferograms
# should be used, else will save results to github repo
os.chdir(ifdirectories[3])
print(ifdirectories[3])

/home/wcc/Harris_etal_2023_InSAR/interferometry/work


In [22]:
from topsApp import TopsInSAR
insar = TopsInSAR(name="topsApp", cmdline=os.path.join(xmldirectories[0], os.listdir(xmldirectories[0])[0]))
insar.configure()

2024-09-24 17:54:53,000 - isce.insar - INFO - ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 


Steps include:
- runPreprocessor
- runComputeBaseline
- verifyDEM
- verifyGeocodeDEM
- runTopo
- runSubsetOverlaps
- runCoarseOffsets
- runCoarseResamp
- runOverlapIfg
- runPrepESD
- runESD
- runRangeCoreg
- runFineOffsets
- runFineResamp
- runIon
- runBurstIfg
- runMergeBursts
- runFilter
- runGeocode
- runDenseOffsets
- runOffsetFilter

I think mine is breaking at runfilter, reinstalled isce2 to check

In [24]:
insar.startup()

None
The currently supported sensors are:  ['SENTINEL1']


In [25]:
insar.runPreprocessor()

Input XML files:  ['/vsizip//home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.zip/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.SAFE/annotation/s1a-iw1-slc-vv-20170601t001818-20170601t001843-016833-01bfc6-004.xml']
Input TIFF files:  ['/vsizip//home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.zip/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.SAFE/measurement/s1a-iw1-slc-vv-20170601t001818-20170601t001843-016833-01bfc6-004.tiff']
Manifest files:  ['/vsizip//home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.zip/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.SAFE/manifest.safe']
MANS:  /home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818

In [26]:
insar.runComputeBaseline()

Estimated burst offset:  0
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Estimated burst offset:  0
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
2024-09-24 17:55:15,233 - isce.topsinsar.runPreprocessor - INFO - 
####################################################################################################
    runComputeBaseline
----------------------------------------------------------------------------------------------------
baseline.IW-1 Bpar at midrange for first common burst = -39.6352604864038
baseline.IW-1 Bpar at midrange for last common burst = -37.32465596659518
baseline.IW-1 Bperp at midrange for first common burst = 76.0456386716753
baseline.IW-1 Bperp at midrange for last common burst = 72.21274432563119
baseline.IW-1 First common burst in reference = 0
baseline.IW-1 First common burst in secondary = 0
baseline.IW-1 Last common burst in reference = 9
baseline.IW-1 Last common burst in secondary = 9
baseline.IW-1 Number of burs

In [27]:
#Verify whether user defined  a dem component.  If not, then download
# SRTM DEM.

#probably where I enter custom DEM, for now not worrying about it

insar.verifyDEM()

command = curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N30W096.SRTMGL1.hgt.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6681k  100 6681k    0     0  6094k      0  0:00:01  0:00:01 --:--:-- 6096k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

command = curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N30W095.SRTMGL1.hgt.zip


100 6573k  100 6573k    0     0  5370k      0  0:00:01  0:00:01 --:--:-- 5374k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

command = curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N30W094.SRTMGL1.hgt.zip


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2190k  100 2190k    0     0  3157k      0 --:--:-- --:--:-- --:--:-- 3155k


command = curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N29W094.SRTMGL1.hgt.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1070k  100 1070k    0     0  1953k      0 --:--:-- --:--:-- --:--:-- 1956k


API open (R): demLat_N29_N31_Lon_W096_W093.dem
API close:  demLat_N29_N31_Lon_W096_W093.dem
Writing geotrans to VRT for demLat_N29_N31_Lon_W096_W093.dem
GDAL open (R): demLat_N29_N31_Lon_W096_W093.dem.vrt
API open (WR): demLat_N29_N31_Lon_W096_W093.dem.wgs84

    << Geoid Correction I2 SRTM>>

Jet Propulsion Laboratory - Radar Science and Engineering


Sampling Geoid at grid points -  Longitude Samples:    33 Latitude Lines:    23
Corner Geoid Heights (m) =  -25.80 -28.35 -27.24 -27.34

Correcting data to geoid height...

At line:      512
At line:     1024
At line:     1536
At line:     2048
At line:     2560
At line:     3072
At line:     3584
At line:     4096
At line:     4608
At line:     5120
At line:     5632
At line:     6144
At line:     6656
At line:     7168
GDAL close: demLat_N29_N31_Lon_W096_W093.dem.vrt
API close:  demLat_N29_N31_Lon_W096_W093.dem.wgs84
API open (R): demLat_N29_N31_Lon_W096_W093.dem.wgs84
API close:  demLat_N29_N31_Lon_W096_W093.dem.wgs84
Writing geotrans

'demLat_N29_N31_Lon_W096_W093.dem.wgs84'

In [28]:
insar.verifyGeocodeDEM()

'demLat_N29_N31_Lon_W096_W093.dem.wgs84'

In [29]:
insar.runTopo()

2024-09-24 17:55:26,003 - isce.zerodop.topozero - WARNING - Default Peg heading set to: -0.22132091182996402


In [ ]:
insar.runSubsetOverlaps()

In [ ]:
insar.runCoarseOffsets()

In [ ]:
insar.runCoarseResamp()

In [ ]:
insar.runOverlapIfg()

In [ ]:
##Prepare for ESD
insar.runPrepESD()

In [ ]:
# run ESD
insar.runESD()

In [ ]:
###Estimate range misregistration
insar.runRangeCoreg()

In [ ]:
###Estimate fine offsets
insar.runFineOffsets()

In [ ]:
###Resample secondary bursts
insar.runFineResamp()

In [ ]:
###calculate ionospheric phase

### need to figure this out
### where should it be place during processing? all that

insar.runIon()

In [ ]:
###Create burst interferograms
insar.runBurstIfg()

In [ ]:
####Merge bursts into single files

#could see if I even need to merge them at all?

insar.runMergeBursts()

In [ ]:
###Filter the interferogram
insar.runFilter()

In [ ]:
#add water mask to coherence and interferogram
#insar.runMaskImages()

In [ ]:
# Unwrap ?
insar.runUnwrapper()

In [ ]:
# Geocode
insar.runGeocode(insar.geocode_list, insar.do_unwrap, insar.geocode_bbox)

In [ ]:
#Dense offsets
insar.runDenseOffsets()

In [ ]:
#Filter offsets
insar.runOffsetFilter()

In [ ]:
#Geocode offsets
insar.runGeocode(insar.off_geocode_list, False, insar.geocode_bbox, True)

In [ ]:
insar.endup()

In [ ]:
# maybe put something here that will move the files generated
# put them into the example directory that Mintpy needs for topsStack



# Working to make it so that the above saves into needed file directories for Mintpy

https://mintpy.readthedocs.io/en/stable/dir_structure/

Essentially just needs another function for:
1. runnings prep_isce.py on the data produced above
2. move all needed data created from ISCE2 into the tsdirectories according to the above link

(this may need to be done in the mintpy environment, so I may end up including this in the time series ipynb, or a short .py to run on it's own)